# 主成分分析PCA

## 背景

  许多机器学习问题涉及每个训练实例的成千上万甚至数百万个特征。正如我们将看到的那样，所有这些特征不仅使**训练变得极其缓慢**，而且还会使找到好的解决方案变得更加困难，大多数训练实例可能彼此之间相距很远。当然，这也意味着新的实例很可能远离任何一个训练实例，导致跟 低维度相比，预测更加不可靠，因为它们基于更大的推测。简而言之， **训练集的维度越高，过拟合的风险就越大**。这个问题通常称为维度的诅咒，所以我们需要用一些方法降低维度。





## 原理

  它识别最靠近数据的超平面，然后将数据投影到上面，如图的样本点为ABC，找到一个超平面y1，使得三个垂线的总和是最小的，最小的轴是对差异性贡献度最高的，其次选择第二条第三条等等。

  <img src="./配图/降维.assets/image-20220421上午112801965.png" alt="image-20220421上午112801965" style="zoom:50%;" />

  ​	使用SVD工具来寻找训练集的主要成分，SVD可以将训练集矩阵 $X$ 分解成 $U\sum V^T$ 的矩阵算法

  <img src="./配图/降维.assets/image-20220421上午113218136.png" alt="image-20220421上午113218136" style="zoom:50%;" />



​	$X_{2d} =XW_d $ 这样，我们的维度就变成和 $W_d$ 一样了。

In [11]:
'''numpy 实现'''
import numpy as np
X = np.random.rand(500,500)
X_centered = X- X.mean(axis=0) # 数据规范化
U, s, Vt = np.linalg.svd(X_centered)
# 提取前两个单位向量
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

# W2就是前两个向量的组合，通过W2进行映射
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [12]:
'''sklearn实现'''
from sklearn.decomposition import PCA
pca = PCA(n_components = 2) # components_属性是Wd的转置
X2D = pca.fit_transform(X)

'''
可解释方差比
该比率表示沿每个成分的数 据集方差的比率。
通过比率可以看出哪部分的信息多
'''
pca.explained_variance_ratio_


array([0.00780391, 0.00768121])

选择正确的维度

与其任意选择要减小到的维度，不如选择相加足够大的方差部分 (例如95%)的维度。当然，如果你是为了数据可视化而降低维度，这 种情况下，需要将维度降低到2或3。

In [13]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1 # d就是需要保存的最少维度
d

305

也可以将n_components设置为0.0到1.0之间的浮点数来表示要保 留的方差率

In [14]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

#这时候查看pca.n_components_
pca.n_components_
#得到的结果和上面的d一致

305

In [15]:
np.sum(pca.explained_variance_ratio_) #输出来确认是否满足我们需要的可解释方差

0.9502812356632158

## PCA压缩

降维过后的训练集占用空间也少了，同样少了维度后是的分类算法的运算速度也加快，也就是通过PCA对数据进行压缩，也可以进行解压缩回到之前的特征数，但是解压后的数据与原始数据有误差，这之前的均方距离就成为重构误差。

In [16]:
pca = PCA(n_components = 154)  # 压缩到154个特征
X_reduced = pca.fit_transform(X)
X_recovered = pca.inverse_transform(X_reduced) # 解压缩回去

## 随机PCA

如果我们的特征数太多，m或n大于500个，并且需要的特征数小于80%，则会自动使用随机PCA算法，该算法可以快速找到前d个 主成分的近似值。

In [29]:
# 如果要强制使用完全的SVD可以设置为‘full’
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X)

## 增量PCA

前面的PCA实现的一个问题是，它们要求整个训练集都放入内存才能运行算法。幸运的是已经开发了增量PCA(IPCA)算法，它们可以使 你把训练集划分为多个小批量，并一次将一个小批量送入IPCA算法。这 对于大型训练集和在线(即在新实例到来时动态运行)应用PCA很有用,请注意，你必须在每个小批量中调用partial_fit()方法，而不是在整个训练集中调用fit()方法。

In [30]:
from sklearn.decomposition import IncrementalPCA
n_batches = 10
inc_pca = IncrementalPCA(n_components=50)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced = inc_pca.transform(X)
X_reduced.shape

/Users/hzh/miniforge3/envs/tfpy39/lib/python3.8/site-packages/sklearn/decomposition/_incremental_pca.py:348: RuntimeWarning: Mean of empty slice.
  self.noise_variance_ = explained_variance[self.n_components_ :].mean()
/Users/hzh/miniforge3/envs/tfpy39/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(500, 50)